In [1]:
# need this import so Term and Sentence
# can talk about each other
from __future__ import annotations

from typing import NamedTuple
import csv
import inspect
import pprint

## Define the `Term` and `Sentence` objects

In [2]:
class Term(NamedTuple):
    """This basically represents all the data needed
    to generate an Anki flaschard for one word / phrase
    in the target language
    """
    text: str  # the word / phrase in the target language.
    translation: str  # the English translation of the word / phrase.
    sentence_ids: list[str]  # list of Tatoeba sentence ids for
                             # the example sentences that go with this term.
                             # This should maybe be a list of ints instead of strs.
    sentences: list[Sentence]  # Objects representing the example sentences.

# This being a named tuple is actually a problem
# bc when it's initialized it doesn't have all of
# its data. Like I want to add the audio id later
# TODO: Make this a regular object
class Sentence:
    tatoeba_id: str  # Tatoeba sentence id for this sentence.
                     # Should maybe be a str.
    text: str  # The text of the sentence.
    translations: list[str] | None  # English translations of this sentence.
                                    # Not all sentences will have translations,
                                    # so this value could be None.
    audio_id: str | None  # Tatoeba audio id for this sentence. Not all
                   # sentences will have audios. If there is no
                   # audio, this will be None. This should maybe be an int.
    parent_term: Term  # A reference to the vocab term that this sentence
                       # serves as an example for. This is useful in the
                       # data generation part. TODO: This actually isn't necessary
    
    def __init__(self, tatoeba_id, text, parent_term):
        self.tatoeba_id = tatoeba_id
        self.text = text
        self.parent_term = parent_term

        self.translations = []
        self.audio_id = None

    def print(self):
        #TODO make this reasonable
        attr_dict = {
            'tatoeba_id': self.tatoeba_id,
            'text': self.text,
            'translations': self.translations,
            'audio_id': self.audio_id
        }
        return pprint.pp(attr_dict)

In [3]:
def process_tsv(process_fn, filepath_str, encoding='utf-8-sig'):
    """Call ROW_FN on each row of a TSV file"""
    with open(filepath_str, newline='', encoding=encoding) as f:
        reader = csv.reader(f, delimiter='\t')
        return process_fn(reader)

## Turn the vocab list `tsv` file into list of `Terms`

In [4]:
def process_vocab_list(tsv_reader, skip_header=True):
    terms = []
    if skip_header:
        header = next(tsv_reader)
    for row in tsv_reader:
        word_de, word_en = row[:2]
        sentence_ids = [s for s in row[2:] if s]
        one_term = Term(word_de, word_en, sentence_ids, [])
        terms.append(one_term)
    return terms

In [5]:
vocab_list_file = './NW_7_Days-and-times - Tageszeiten.tsv'
terms = process_tsv(process_vocab_list, vocab_list_file)

In [6]:
terms

[Term(text='der Abend, die Abende', translation='evening', sentence_ids=['998'], sentences=[]),
 Term(text='das Abendessen, die Abendessen', translation='dinner', sentence_ids=['341418'], sentences=[]),
 Term(text='am Morgen', translation='in the morning', sentence_ids=['347433'], sentences=[]),
 Term(text='jemanden an|rufen', translation='to call someone', sentence_ids=['873078'], sentences=[]),
 Term(text='def Apfel, die Äpfel', translation='apple', sentence_ids=['367095'], sentences=[]),
 Term(text='etwas auf|räumen', translation='to clean up something; to tidy up something', sentence_ids=['618818'], sentences=[]),
 Term(text='aus|gehen', translation='to go out', sentence_ids=['882822'], sentences=[]),
 Term(text='auf|stehen', translation='to get up', sentence_ids=['365619'], sentences=[]),
 Term(text='etwas ein|kaufen', translation='to buy/shop for something', sentence_ids=['3567432'], sentences=[]),
 Term(text='fast immer', translation='almost always', sentence_ids=['2571282'], se

## Now add the sentence data to the `Terms`

In [7]:
def add_sentences(tsvreader, terms, validate=True):
    term_by_id = {}
    for t in terms:
        for sentence_id in t.sentence_ids:
            term_by_id[sentence_id] = t
    for row in tsvreader:
        sentence_id, text = row[0], row[2]
        if sentence_id in term_by_id:
            term = term_by_id[sentence_id]
            sentence = Sentence(sentence_id, text, term)
            term.sentences.append(sentence)
    if validate:
        for t in terms:
            for s in t.sentences:
                assert s.tatoeba_id
                assert s.tatoeba_id in t.sentence_ids
                assert s.text
    return

In [8]:
sentences_file = 'deu_sentences.tsv'

In [9]:
process_tsv(lambda x: add_sentences(x, terms, True), sentences_file)

In [10]:
terms

[Term(text='der Abend, die Abende', translation='evening', sentence_ids=['998'], sentences=[<__main__.Sentence object at 0x00000216D160CE60>]),
 Term(text='das Abendessen, die Abendessen', translation='dinner', sentence_ids=['341418'], sentences=[<__main__.Sentence object at 0x00000216D160DC10>]),
 Term(text='am Morgen', translation='in the morning', sentence_ids=['347433'], sentences=[<__main__.Sentence object at 0x00000216D160CCE0>]),
 Term(text='jemanden an|rufen', translation='to call someone', sentence_ids=['873078'], sentences=[<__main__.Sentence object at 0x00000216D160E450>]),
 Term(text='def Apfel, die Äpfel', translation='apple', sentence_ids=['367095'], sentences=[<__main__.Sentence object at 0x00000216D160DFA0>]),
 Term(text='etwas auf|räumen', translation='to clean up something; to tidy up something', sentence_ids=['618818'], sentences=[<__main__.Sentence object at 0x00000216D160E1E0>]),
 Term(text='aus|gehen', translation='to go out', sentence_ids=['882822'], sentences=[<

## Add translation data to the `Terms`
Provided that there is translation data

In [11]:
def add_translations(tsvreader, terms, limit=3):
    sentence_by_id = {}
    for t in terms:
        for s in t.sentences:
            sentence_by_id[s.tatoeba_id] = s
    for row in tsvreader:
        sentence_id = row[0]
        translation = row[3]
        if sentence_id in sentence_by_id:
            s = sentence_by_id[sentence_id]
            if len(s.translations) < limit:
                s.translations.append(translation)
    return

In [12]:
translations_file = 'Sentence pairs in German-English - 2024-12-13.tsv'

In [13]:
process_tsv(lambda x: add_translations(x, terms), translations_file)

## Add audio data to the `Terms`

In [14]:
def add_audio_ids(tsvreader, terms):
    sentence_by_id = {}
    for t in terms:
        for s in t.sentences:
            sentence_by_id[s.tatoeba_id] = s
    for row in tsvreader:
        sentence_id = row[0]
        audio_id = row[1]
        if sentence_id in sentence_by_id:
            s = sentence_by_id[sentence_id]
            s.audio_id = audio_id
            # Delete the key in case the data file
            # has multiple audio ids for this sentence
            sentence_by_id.pop(sentence_id)
    return

In [15]:
audio_id_file = 'sentences_with_audio.csv'

In [16]:
process_tsv(lambda x: add_audio_ids(x, terms), audio_id_file)

In [17]:
terms[0].sentences[0].print()

{'tatoeba_id': '998',
 'text': 'Ich dusche gewöhnlich abends.',
 'translations': ['I usually take a shower in the evening.',
                  'I usually shower at night.'],
 'audio_id': '634'}


In [18]:
no_audios = [t for t in terms if any(map(lambda s: not s.audio_id, t.sentences))]

In [19]:
no_audios

[Term(text='am Morgen', translation='in the morning', sentence_ids=['347433'], sentences=[<__main__.Sentence object at 0x00000216D160CCE0>]),
 Term(text='etwas ein|kaufen', translation='to buy/shop for something', sentence_ids=['3567432'], sentences=[<__main__.Sentence object at 0x00000216D160E630>]),
 Term(text='fast immer', translation='almost always', sentence_ids=['2571282'], sentences=[<__main__.Sentence object at 0x00000216D13D12E0>]),
 Term(text='die Flöte, die Flöten', translation='flute', sentence_ids=['11932738'], sentences=[<__main__.Sentence object at 0x00000216D160E870>]),
 Term(text='das Mittagessen, die Mittagessen', translation='lunch', sentence_ids=['10513933'], sentences=[<__main__.Sentence object at 0x00000216D160E780>]),
 Term(text='das Müsli, die Müslis', translation='granola', sentence_ids=['10919429'], sentences=[<__main__.Sentence object at 0x00000216D160E7E0>]),
 Term(text='der Nachmittag, die Nachmittage', translation='afternoon', sentence_ids=['989712', '1807

In [20]:
no_audios[0].sentences[0].print()

{'tatoeba_id': '347433',
 'text': 'Könnte ich am Morgen duschen?',
 'translations': ['May I take a shower in the morning?'],
 'audio_id': None}
